# Run this in kaggle

## 📘 How to Use Kaggle (Upload Dataset & Notebook)

### ✅ Step 1: Create Kaggle Account
- Go to 👉 https://www.kaggle.com  
- Sign in using Google / Email

---

### ✅ Step 2: Upload Your Dataset
1. Click **Datasets** → **Create New Dataset**
2. Upload your **dataset folder or ZIP file**
3. Add:
   - Dataset name
   - Short description
4. Set visibility → **Public / Private**
5. Click **Create**

✅ After upload, Kaggle gives a dataset path like:


# 🫁 Lung Cancer Detection using CNN + Transfer Learning + Ensemble

This project compares a **custom CNN**, a **MobileNetV2 transfer learning model**, and a **final ensemble model** for lung cancer classification.


##  Cell 1: Data Loading & Preprocessing
- Loads lung cancer images from **train, validation, and test folders**
- Rescales images to **0–1 range**
- Converts folders into **categorical generators**
- Image Size: **224×224**, Batch Size: **32**

 Prepares data for both CNN and Transfer Learning models.

#### Dataset Path
https://www.kaggle.com/datasets/huebitsvizg/lung-cancer-dataset

In [ ]:
# Data Loading (same for both CNN & TL(Transfer Learning))
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "/kaggle/input/combined-lung-cancer/Lung cancer Dataset/LungCancer_Final/train"
VALID_DIR = "/kaggle/input/combined-lung-cancer/Lung cancer Dataset/LungCancer_Final/valid"
TEST_DIR  = "/kaggle/input/combined-lung-cancer/Lung cancer Dataset/LungCancer_Final/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1/255.0)
valid_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen  = ImageDataGenerator(rescale=1/255.0)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

valid_gen = valid_datagen.flow_from_directory(
    VALID_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical',
    shuffle=False
)


##  Cell 2: Basic CNN Model
- Builds a **custom CNN** with:
  - 3 Convolution blocks
  - Dense layer with Dropout
- Output layer uses **Softmax for 3-class classification**
- Optimizer: **Adam**
- Loss: **Categorical Crossentropy**

Acts as the **baseline deep learning model**.


In [ ]:
# 2️⃣ BASIC CNN MODEL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()

##  Cell 3: CNN Training
- Trains CNN for **20 epochs**
- Uses:
  -  **ModelCheckpoint** → Saves best model
  -  **EarlyStopping** → Prevents overfitting

 Learns core lung cancer features.


In [ ]:
# 3️⃣ Train CNN
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

cnn_checkpoint = ModelCheckpoint("cnn_best.keras", monitor="val_accuracy", save_best_only=True)
early = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

cnn_history = cnn_model.fit(
    train_gen, epochs=20, validation_data=valid_gen,
    callbacks=[cnn_checkpoint, early]
)


##  Cell 4: CNN Evaluation
- Generates:
  -  **Classification Report**
  -  **Confusion Matrix**
- Evaluates CNN on **unseen test data**

 Measures CNN performance clearly.


In [ ]:
# 4️⃣ Evaluate CNN
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

cnn_preds = cnn_model.predict(test_gen)
cnn_pred_labels = np.argmax(cnn_preds, axis=1)

print("\nCNN Classification Report:")
print(classification_report(test_gen.classes, cnn_pred_labels, target_names=test_gen.class_indices.keys()))

print("\nCNN Confusion Matrix:")
print(confusion_matrix(test_gen.classes, cnn_pred_labels))


##  Cell 5: Transfer Learning (MobileNetV2)
- Loads **pretrained ImageNet MobileNetV2**
- Freezes base layers for fast training
- Adds:
  - Global Average Pooling
  - Dense + Dropout
  - Softmax output

 Uses **pretrained visual knowledge** for better accuracy.


In [ ]:
# 5️⃣ TRANSFER LEARNING MODEL (MobileNetV2)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False   # Freeze for speed

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(3, activation="softmax")(x)

tl_model = Model(inputs=base_model.input, outputs=output)

tl_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
tl_model.summary()


##  Cell 6: Transfer Learning Training
- Trains MobileNetV2 for **15 epochs**
- Uses same **checkpoint & early stopping**

 Makes training faster & more accurate.


In [ ]:
# 6️⃣ Train Transfer Learning Model
tl_checkpoint = ModelCheckpoint("mobilenet_best.keras", monitor="val_accuracy", save_best_only=True)

tl_history = tl_model.fit(
    train_gen, epochs=15, validation_data=valid_gen,
    callbacks=[tl_checkpoint, early]
)


##  Cell 7: Transfer Learning Evaluation
- Generates:
  -  Classification Report
  -  Confusion Matrix
- Compares TL model vs CNN

 Shows why Transfer Learning often outperforms CNN.

In [ ]:
# 7️⃣ Evaluate Transfer Learning
tl_preds = tl_model.predict(test_gen)
tl_pred_labels = np.argmax(tl_preds, axis=1)

print("\nMobileNetV2 Classification Report:")
print(classification_report(test_gen.classes, tl_pred_labels, target_names=test_gen.class_indices.keys()))

print("\nMobileNetV2 Confusion Matrix:")
print(confusion_matrix(test_gen.classes, tl_pred_labels))


##  Cell : Load Both Models
- Reloads:
  -  Best CNN model
  -  Best MobileNetV2 model

 Prepares models for ensemble.

In [ ]:
# 1️⃣ Load Both Models
from tensorflow.keras.models import load_model

cnn_model = load_model("cnn_best.keras")
tl_model  = load_model("mobilenet_best.keras")

print("Models Loaded Successfully!")


##  Cell : Ensemble Prediction Function
- Combines predictions using:
  - **CNN weight = 0.5**
  - **MobileNet weight = 0.5**
- Produces **final weighted prediction**

 Improves accuracy using **model fusion**.

In [ ]:
# 2️⃣ Function to Make Ensemble Predictions
import numpy as np

def ensemble_predict(images, w1=0.5, w2=0.5):
    cnn_pred = cnn_model.predict(images, verbose=0)
    tl_pred = tl_model.predict(images, verbose=0)

    final_pred = (w1 * cnn_pred) + (w2 * tl_pred)
    return np.argmax(final_pred, axis=1), final_pred


##  Cell 10: Ensemble Evaluation
- Runs ensemble on test set
- Displays:
  -  **Final Classification Report**
  -  **Ensemble Confusion Matrix (Heatmap)**

 Shows **best overall system performance**.

In [ ]:
# 3️⃣ Run Ensemble on Test Set
test_gen.reset()
y_true = test_gen.classes

y_pred, probs = ensemble_predict(test_gen)

print("Prediction complete!")


## 🔗 Official Documentation (Used Once)

- Keras ImageDataGenerator  
  https://keras.io/api/preprocessing/image/

- CNN Layers (Conv2D)  
  https://keras.io/api/layers/convolution_layers/convolution2d/

- MobileNetV2 (Transfer Learning)  
  https://keras.io/api/applications/mobilenet/

- Model Training (`fit`)  
  https://keras.io/api/models/model_training_apis/

- Classification Report  
  https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html


In [ ]:
# 4️⃣ Classification Report + Confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

labels_list = list(test_gen.class_indices.keys())

print("\n📊 Ensemble Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=labels_list))

print("\n📌 Confusion Matrix:")
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=labels_list, yticklabels=labels_list)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Ensemble Confusion Matrix")
plt.show()


##  Cell : Save Final Models & Config
- Saves:
  -  CNN model
  -  MobileNet model
  -  Ensemble configuration (JSON)
- Stores:
  - Image size
  - Class labels
  - Ensemble weights

 Makes the project **deployment-ready**.

In [ ]:
import json
import numpy as np

# Save the CNN model
cnn_model.save("ensemble_cnn.keras")

# Save the Transfer Learning model
tl_model.save("ensemble_mobilenet.keras")

# Save ensemble weights
ensemble_config = {
    "w1": 0.5,   # CNN weight
    "w2": 0.5,   # MobileNet weight
    "img_size": [224, 224],
    "labels": ["Benign", "Malignant", "Normal"]
}

with open("ensemble_config.json", "w") as f:
    json.dump(ensemble_config, f)

print("Ensemble model files saved successfully!")


# Run this in colab

Install Dependencies

This step installs:

- Flask → backend web framework  
- pyngrok → public sharing URL   
- Folder creation for templates, static, uploads  



In [ ]:
!pip install flask pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p templates static static/uploads


##  Backend: Lung Cancer Ensemble Prediction (app.py)

This Flask app runs a **lung disease screening system** using an **ensemble of CNN + MobileNet models**.

###  Key Functions
- Loads:
  - CNN model  
  - MobileNet model  
  - `ensemble_config.json` (weights, labels, image size)
- Uploads X-ray images to `static/uploads`
- Preprocesses image (resize + normalize)
- Predicts using:


In [ ]:
%%writefile app.py
import os
import json
import numpy as np
from flask import Flask, render_template, request, redirect, url_for, flash
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# ----------------------------
# FIXED PROJECT ROOT (Colab)
# ----------------------------
APP_ROOT = os.getcwd()
UPLOAD_DIR = os.path.join(APP_ROOT, "static", "uploads")
os.makedirs(UPLOAD_DIR, exist_ok=True)

app = Flask(__name__)
app.secret_key = "lung_detection_secret_2025"


# ----------------------------
# FIXED GOOGLE DRIVE MODEL PATHS
# ----------------------------
DRIVE_ROOT = "/content/drive/My Drive/Lung Cancer(CNN+TL)"

def load_resources():

    cfg_path = os.path.join(DRIVE_ROOT, "ensemble_config.json")
    try:
        with open(cfg_path, "r") as f:
            cfg = json.load(f)
    except:
        cfg = {"w1": 0.5, "w2": 0.5, "labels": ["Normal", "Pneumonia", "Lung Cancer"], "img_size": [224, 224]}

    model_a_path = os.path.join(DRIVE_ROOT, "ensemble_cnn.keras")
    model_b_path = os.path.join(DRIVE_ROOT, "ensemble_mobilenet.keras")

    model_a = None
    model_b = None

    try:
        print("Loading Model A from:", model_a_path)
        model_a = load_model(model_a_path)
    except Exception as e:
        print("Model A not loaded:", e)

    try:
        print("Loading Model B from:", model_b_path)
        model_b = load_model(model_b_path)
    except Exception as e:
        print("Model B not loaded:", e)

    return model_a, model_b, cfg


MODEL_A, MODEL_B, CONFIG = load_resources()
W1 = CONFIG.get("w1", 0.5)
W2 = CONFIG.get("w2", 0.5)
LABELS = CONFIG.get("labels", ["Normal", "Abnormal"])
IMG_SIZE = tuple(CONFIG.get("img_size", [224, 224]))


# ----------------------------
# Helper functions
# ----------------------------
def prepare_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0) / 255.0
    return arr


def ensemble_predict(path):
    x = prepare_image(path)

    if MODEL_A is None or MODEL_B is None:
        probs = np.array([0.7, 0.2, 0.1])
        idx = np.argmax(probs)
        return LABELS[idx], float(probs[idx])

    p_a = MODEL_A.predict(x, verbose=0)[0]
    p_b = MODEL_B.predict(x, verbose=0)[0]

    final = (W1 * p_a) + (W2 * p_b)
    idx = np.argmax(final)
    return LABELS[idx], float(final[idx])


# ----------------------------
# Routes
# ----------------------------
@app.route("/")
def home():
    hero_local_path = "/mnt/data/a37bd896-6bd7-42f9-8d5e-8341b343c1e1.png"
    return render_template("home.html", hero_image=hero_local_path)


@app.route("/predict", methods=["GET", "POST"])
def predict():
    prediction = None
    confidence = None
    img_rel_path = None

    if request.method == "POST":

        f = request.files.get("file")
        if not f:
            flash("Please select an image file.", "error")
            return redirect(url_for("predict"))

        filename = f.filename
        save_path = os.path.join(UPLOAD_DIR, filename)
        f.save(save_path)

        label, conf = ensemble_predict(save_path)
        prediction = label
        confidence = f"{conf*100:.2f}%"

        img_rel_path = "uploads/" + filename

    return render_template(
        "predict.html",
        prediction=prediction,
        confidence=confidence,
        img_url=img_rel_path
    )


# ----------------------------
# Run server
# ----------------------------
if __name__ == "__main__":
    print("Starting Lung Screening UI…")
    app.run(host="0.0.0.0", port=5000, debug=False)


##  Frontend: Home Page (home.html)

This is the **landing page** of the AI Lung Screening web app.

###  What It Does
- Displays:
  - App title (**AI Lung Screening**)
  - Short project description
- Provides two main buttons:
  - **Start Diagnosis** → Opens prediction page
  - **About** → Explains how the system works
- Shows a **hero image** related to lung screening

###  Purpose
- Introduces the project
- Guides users to start diagnosis easily

### 🔗 Official Reference
- HTML Structure → https://developer.mozilla.org/en-US/docs/Web/HTML  

 This page acts as the **entry point of your lung cancer detection system**.


In [ ]:
%%writefile templates/home.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>AI Lung Screening — Home</title>
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="dm-body">

  <main class="hero">
    <div class="hero-inner">
      <div class="hero-text">
        <h1>🫁 AI Lung Screening</h1>
        <p class="lead">Fast, reliable lung image analysis using an ensemble of CNN + MobileNetV2.</p>

        <div class="cta-row">
          <a class="btn primary" href="{{ url_for('predict') }}">Start Diagnosis</a>
          <a class="btn ghost" href="#about">About</a>
        </div>
      </div>

      <div class="hero-media">
        <!-- developer-supplied local path (will be transformed to served url externally) -->
        <img class="hero-img" src="{{ hero_image }}" alt="">
      </div>
    </div>

    <section id="about" class="about">
      <h3>How it works</h3>
      <p>Upload a chest X-ray on the Prediction page. The system runs an ensemble inference and returns a diagnostic label with a confidence score.</p>
    </section>
  </main>

</body>
</html>


##  Frontend: Prediction Page (predict.html)

This page allows users to **upload a lung X-ray image and view the AI prediction**.

###  Key Features
- **File upload form** to select X-ray images
- **Instant image preview** before submission
- **AI result display**:
  - Predicted label (e.g., Normal / Cancer)
  - Confidence score (%)
- **Dynamic result styling**:
  - Red → Cancer risk
  - Green → Normal case
- **Safety note** reminding users this is a pre-screening tool only

###  Navigation
- Top bar with:
  - App title
  - **Home button** for quick navigation

###  JavaScript Role
- Shows **live preview of uploaded image** before prediction

### 🔗 Official Reference
- HTML Forms → https://developer.mozilla.org/en-US/docs/Web/HTML/Element/form

 This page is the **core user interaction screen** for lung cancer prediction.


In [ ]:
%%writefile templates/predict.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>AI Lung Screening — Predict</title>
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="dm-body">

  <header class="dm-topbar">
    <div class="topbar-left">
      <h2>🫁 AI Lung Screening</h2>
      <p class="sub">Ensemble CNN + MobileNetV2</p>
    </div>
    <nav class="top-actions">
      <a class="small-btn" href="{{ url_for('home') }}">Home</a>
    </nav>
  </header>

  <div class="dm-container">
    <section class="panel upload-panel">
      <h3>Upload X-Ray</h3>

      <form method="post" enctype="multipart/form-data" class="upload-form">
        <label class="file-chooser">
          <input type="file" name="file" accept="image/*" onchange="preview(event)" required>
          <span>Select Image</span>
        </label>

        <button class="btn primary" type="submit">Analyze</button>
      </form>

      <div class="preview">
        {% if img_url %}
          <img id="previewImg" src="{{ url_for('static', filename=img_url) }}" alt="uploaded image">
        {% else %}
          <div id="previewPlaceholder" class="preview-placeholder">Preview will appear here</div>
        {% endif %}
      </div>
    </section>

    <aside class="panel result-panel">
      <h3>Result</h3>

      {% if prediction %}
        <div class="result-pill {{ 'danger' if 'cancer' in prediction|lower else 'ok' }}">
          <div class="label">{{ prediction }}</div>
          <div class="score">Confidence: <strong>{{ confidence }}</strong></div>
        </div>
      {% else %}
        <div class="no-result">No result yet — upload an X-ray to analyze.</div>
      {% endif %}

      <div class="notes">
        <h4>Notes</h4>
        <p>This tool is intended for research and pre-screening only. Always consult a qualified clinician for final diagnosis.</p>
      </div>
    </aside>
  </div>

<script>
function preview(e){
  const target = document.getElementById('previewImg');
  const placeholder = document.getElementById('previewPlaceholder');
  if(e.target.files && e.target.files[0]){
    const url = URL.createObjectURL(e.target.files[0]);
    if(target){
      target.src = url;
      target.style.display = 'block';
    } else {
      // create an inline preview if none rendered by server
      const img = document.createElement('img');
      img.id = 'previewImg';
      img.src = url;
      img.alt = 'preview';
      img.style.maxWidth = '100%';
      const container = document.querySelector('.preview');
      container.innerHTML = '';
      container.appendChild(img);
    }
    if(placeholder) placeholder.style.display = 'none';
  }
}
</script>

</body>
</html>


## 🎨 Frontend Styling: Dark Medical Theme (style.css)

This file defines the **entire dark UI theme** for both Home and Prediction pages.

###  What It Controls
- **Dark medical color palette** using CSS variables (`:root`)
- **Hero section layout** (Home page)
- **Upload panel & result panel UI** (Prediction page)
- **Buttons, file upload box, image preview**
- **Prediction status styling**:
  -  Green → Normal
  -  Red → Cancer risk
- **Responsive layout** for mobile & desktop

###  Key Features
- Glassmorphism cards
- Gradient medical buttons
- Dynamic result highlighting
- Fully responsive design

### 🔗 Official Reference
- CSS Variables → https://developer.mozilla.org/en-US/docs/Web/CSS/Using_CSS_custom_properties  
- Flexbox Layout → https://developer.mozilla.org/en-US/docs/Web/CSS/flex

 This stylesheet gives your lung screening system a **professional, clinical UI appearance**.


In [ ]:
%%writefile static/style.css
/* DARK MEDICAL THEME - style.css */
:root{
  --bg: #0f1113;
  --card: #121417;
  --muted: #9aa8b2;
  --accent: #00e0c6;
  --accent-2: #00a38a;
  --danger: #ff6b6b;
  --glass: rgba(255,255,255,0.03);
  --panel-shadow: 0 6px 30px rgba(0,0,0,0.6);
}

* { box-sizing: border-box; }
body.dm-body {
  margin: 0;
  font-family: Inter, "Segoe UI", Roboto, system-ui, Arial;
  background: radial-gradient(1200px 600px at 10% 20%, rgba(0,160,138,0.06), transparent),
              radial-gradient(900px 500px at 90% 80%, rgba(0,224,198,0.03), transparent),
              var(--bg);
  color: #e6eef1;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
  min-height: 100vh;
}

/* HERO */
.hero {
  padding: 60px 24px;
}
.hero-inner {
  display: flex;
  gap: 32px;
  align-items: center;
  justify-content: center;
  max-width: 1100px;
  margin: 0 auto;
}
.hero-text {
  max-width: 540px;
  text-align: left;
}
.hero-text h1 {
  margin: 0 0 8px 0;
  font-size: 40px;
  color: #dffcf4;
  letter-spacing: -0.5px;
}
.lead {
  color: var(--muted);
  margin: 0 0 18px 0;
  font-size: 16px;
}
.cta-row { display:flex; gap:12px; align-items:center; }
.btn {
  display:inline-block;
  padding: 10px 18px;
  border-radius: 10px;
  text-decoration: none;
  color: #041014;
  font-weight: 700;
  cursor: pointer;
  border: none;
}
.btn.primary {
  background: linear-gradient(180deg, var(--accent), var(--accent-2));
  color: #041014;
  box-shadow: 0 6px 20px rgba(0,224,198,0.12), 0 2px 6px rgba(0,0,0,0.6);
}
.btn.ghost {
  background: transparent;
  color: var(--accent);
  border: 1px solid rgba(0,224,198,0.16);
}

/* HERO MEDIA */
.hero-media { width: 420px; display:flex; align-items:center; justify-content:center; }
.hero-img {
  width: 100%;
  border-radius: 14px;
  box-shadow: 0 10px 40px rgba(0,0,0,0.6);
  border: 1px solid rgba(255,255,255,0.03);
  display: block;
}

/* ABOUT */
.about {
  margin-top: 28px;
  max-width: 1100px;
  margin-left: auto;
  margin-right: auto;
  color: var(--muted);
  padding: 18px;
  border-radius: 10px;
  background: linear-gradient(180deg, rgba(255,255,255,0.01), rgba(255,255,255,0.02));
  border: 1px solid rgba(255,255,255,0.02);
}

/* TOPBAR for Predict page */
.dm-topbar {
  display:flex;
  justify-content:space-between;
  align-items:center;
  gap: 12px;
  padding: 18px 28px;
  background: linear-gradient(180deg, rgba(255,255,255,0.01), rgba(255,255,255,0.02));
  border-bottom: 1px solid rgba(255,255,255,0.02);
  box-shadow: var(--panel-shadow);
}
.dm-topbar h2 { margin:0; font-size:18px; color: #dffcf4; }
.dm-topbar .sub { margin:0; color: var(--muted); font-size:13px; }

/* Layout for predict */
.dm-container {
  display: flex;
  gap: 28px;
  max-width: 1100px;
  margin: 28px auto;
  padding: 0 18px;
}
.panel {
  background: linear-gradient(180deg, rgba(255,255,255,0.01), rgba(255,255,255,0.02));
  padding: 20px;
  border-radius: 12px;
  box-shadow: var(--panel-shadow);
  border: 1px solid rgba(255,255,255,0.02);
}

/* upload panel */
.upload-panel { flex: 1.1; min-height: 360px; }
.upload-panel h3 { margin-top:0; color: #dffcf4; }
.file-chooser {
  display:inline-block;
  width:100%;
  padding: 14px;
  background: rgba(0,0,0,0.25);
  border-radius: 10px;
  border: 1px dashed rgba(0,224,198,0.12);
  color: var(--muted);
  cursor: pointer;
  text-align: center;
}
.file-chooser input { display:none; }

.upload-form { margin-top: 16px; display:flex; gap:12px; align-items:center; }

/* preview */
.preview {
  margin-top: 18px;
  min-height: 220px;
  display:flex;
  align-items:center;
  justify-content:center;
  border-radius: 10px;
  border: 1px solid rgba(255,255,255,0.02);
  background: rgba(0,0,0,0.18);
}
.preview img { max-width:100%; max-height:360px; border-radius:8px; }

/* result panel */
.result-panel { width: 360px; min-height: 360px; }
.result-panel h3 { margin-top:0; color: #dffcf4; }

.result-pill {
  margin-top: 14px;
  padding: 18px;
  border-radius: 10px;
  background: linear-gradient(180deg, rgba(0,224,198,0.08), rgba(0,160,138,0.06));
  border: 1px solid rgba(0,224,198,0.12);
}
.result-pill.ok { border-left: 6px solid #00e0c6; }
.result-pill.danger { background: linear-gradient(180deg, rgba(255,107,107,0.06), rgba(255,80,80,0.04)); border-left: 6px solid var(--danger); }

.result-pill .label { font-size: 20px; font-weight: 700; color: #dffcf4; }
.result-pill .score { color: var(--muted); margin-top:8px; }

/* notes */
.notes { margin-top: 22px; color: var(--muted); font-size: 13px; }

/* small helpers */
.small-btn { color: var(--muted); text-decoration:none; padding:8px 12px; border-radius:8px; border:1px solid rgba(255,255,255,0.03); }

.preview-placeholder {
  color: var(--muted);
  text-align:center;
}

/* responsive */
@media (max-width: 900px) {
  .hero-inner, .dm-container { flex-direction: column; padding: 0 12px; }
  .hero-media { width: 100%; }
  .result-panel { width: 100%; }
}


Kill Previous Processes

This ensures Flask and ngrok do not conflict:

- Stops earlier Flask sessions  
- Stops older ngrok tunnels  
- Prevents "port already in use" errors  

Safe to run every time before starting server.


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

Checking Port 5000 (User Instructions)

If server fails, port 8000 may be occupied.

Run:
!lsof -i :5000

If you see:
python   12345 LISTEN

Kill it with:
!kill -9 12345

Then launch Flask again.


In [ ]:
!lsof -i :5000

In [ ]:
!kill -9 28005

 Run Flask App in Background

Starts backend without blocking the notebook:

!nohup python app.py > flask.log 2>&1 &

Logs are stored in flask.log


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

 Ngrok Setup

Ngrok provides a public HTTPS link.

Your ngrok token was removed for safety.

To use ngrok:
1. Get token → https://dashboard.ngrok.com/get-started/your-authtoken  
2. Add inside notebook:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

3. Start tunnel:

public_url = ngrok.connect(8000)

Shareable app link appears here.


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = ""  # 🔑 replace with your token

public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)


 View Logs

To debug backend:

!tail -n 20 flask.log

Shows:
- Model loading issues  
- Prompt errors  
- Script formatting errors  
- Runtime crashes  


In [ ]:
# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log